In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 19
### Team Member Names: Steven Chen, Jolene Huang, Haadi Khan
### Team Strategy Chosen: RISKY

Before implementing our risky portfolio strategy, we must first define constants and variables:

In [2]:
# Constants
TEAM_NUMBER = 19
TICKERS_FILE_NAME = "Tickers.csv"
STARTING_VALUE = 750000
CURRENCIES = ["USD", "CAD"]
USD_TO_CAD = "USDCAD=x"
START_DATE ="2023-01-01"
END_DATE ="2023-10-31"
MIN_DAYS = 18
MIN_VOLUME = 150000
NUM_STOCKS = 10
STOCKS_WEIGHTING = pd.Series([0.2, 0.2, 0.2, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])
MARKET_INDEX = "^GSPC"
TRADING_FEE = 4.95

We are using 10 stocks in our portfolio for this strategy and we are weighting each stock differently based on their risk. The constant, STOCK_WEIGHTING is a Series that represents the weight we assign to each of the stocks in our portfolio, in decreasing order of risk. We want to maximize the weightings of the most risky stocks for this strategy while complying with the requirements of a 20% maximum weighting and each stock making up a minimum of (100/(2n))%, where n is the number of stocks in our portfolio. Therefore, the top three riskiest stocks will be weighted 20%, the fourth riskiest stock will be weighted 10%, and the rest of the stocks will each be weighted 5% in our portfolio of ten stocks in total.

We must then turn the given csv file, “Tickers.csv” into a list of strings as the ticker names.

In [3]:
# Get list of all tickers from csv file
symbol_list = pd.read_csv(TICKERS_FILE_NAME, header=None)[0].to_list()
print(symbol_list)

['AAPL', 'ABBV', 'ABT', 'ACN', 'AGN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CELG', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTN', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


Since it is not guaranteed that all of these tickers are valid, we now create a function called get_valid_stocks that filters out stocks that do not meet our requirements. By doing this, we can eliminate stocks that do not have an average monthly volume of at least 150,000 shares and those that are not denominated in USD or CAD.

Firstly, we downloaded the stock data from the Yahoo Finance API. stocks.tickers is a property of the stocks object which holds the details of individual stock tickers. The stock_data dictionary is initialized to store the historical data of valid stocks. Using a for loop, we iterate through each symbol in stock_tickers and filter the stock data according to the criteria mentioned. We also use this function to filter out the months with less than 18 trading days.

If the stock passes all these requirements, its historical data is stored in the stock_data dictionary with the symbol as the key. This filtering process can be used in financial analysis as we want to make sure that chosen stocks have sufficient liquidity, are traded in the desired currencies, and include enough data to be analyzed adequately.


In [4]:
# Get only valid stock data
stocks = yf.Tickers(symbol_list)
stock_tickers = stocks.tickers
stock_data = {}

def get_valid_stocks():
  for i, symbol in enumerate(stock_tickers):
    try:
      ticker = stock_tickers[symbol]

      # Filter out stocks that aren't in CAD OR USD
      if ticker.fast_info["currency"] not in CURRENCIES:
        continue

      history = ticker.history(start=START_DATE, end=END_DATE)

      # Filter out months without at least 18 trading days
      history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])

      # Skip stock if not enough average monthly volume
      if history.Volume.mean() < MIN_VOLUME:
        continue

      stock_data[symbol] = history

    except Exception as e:
      print(f"{type(e).__name__}: {e}")
      continue

get_valid_stocks()


/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/

KeyError: 'currentTradingPeriod'


/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/

KeyError: 'currentTradingPeriod'


/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/

KeyError: 'currentTradingPeriod'


/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/

KeyError: 'currentTradingPeriod'


/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/pl3ldchs7kzb9cbkl7w2rqtr0000gn/T/ipykernel_58389/2072376713.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  history = history.assign(YearMonth=history.index.to_period('M')).groupby('YearMonth').filter(lambda x: len(x) >= MIN_DAYS).drop(columns=['YearMonth'])
/var/folders/bs/

Weekly returns are calculated by taking the percentage change between the current week's closing price and the previous week's closing price. We chose to observe the weekly returns instead of daily returns as there will be less variability in percentage changes over a week compared to a day. We created a function called get_weekly_returns that calculates these values and stores them into the weekly_returns dataframe.

In [5]:
# Get weekly percent returns of stocks
def get_weekly_returns():
  weekly_returns = pd.DataFrame()
  symbols = list(stock_data.keys())

  for symbol in symbols:
    # Resample to weekly stock data and get percent returns
    stock_weekly_returns = stock_data[symbol].Close.resample("W").last().pct_change().dropna()
    weekly_returns = pd.concat([weekly_returns, stock_weekly_returns], axis=1)

  # Set column names to stock symbols
  weekly_returns.columns = symbols

  return weekly_returns

weekly_returns = get_weekly_returns()
display(weekly_returns.head(5))

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
2023-01-15 00:00:00-05:00,0.039654,-0.068988,0.015080,0.052316,-0.002169,0.139870,0.037224,0.005305,0.023830,0.031477,...,0.042483,0.029518,0.051049,0.041821,0.028598,0.021980,-0.000878,0.005754,0.017882,0.022025
2023-01-22 00:00:00-05:00,0.023078,-0.026107,-0.006079,-0.005919,-0.030120,-0.008867,-0.026708,-0.034418,-0.039171,-0.007881,...,0.022122,0.003202,0.039739,0.011368,0.003827,-0.033574,-0.005821,-0.021432,-0.022070,-0.018382
2023-01-29 00:00:00-05:00,0.058461,-0.022127,-0.025439,-0.011409,0.013767,0.051311,0.136609,0.021329,0.047267,0.015082,...,0.085435,0.009415,0.230798,-0.002459,0.025118,0.012948,-0.001377,-0.030049,0.022231,0.051657
2023-02-05 00:00:00-05:00,0.058727,-0.007383,0.016735,0.043099,-0.072320,0.011248,0.038013,-0.024435,0.027645,-0.022235,...,0.012144,0.025993,0.064056,0.007747,0.010939,0.047644,-0.028865,0.039034,0.052556,0.009005
2023-02-12 00:00:00-05:00,-0.021097,0.047176,-0.032472,-0.018187,0.043745,-0.055905,0.002180,0.033396,-0.023332,0.009414,...,-0.044660,0.004415,-0.091499,-0.047869,0.006384,-0.035657,0.047095,-0.022778,-0.033601,-0.012170


Standard deviation measures the spread of data compared to the mean of the dataset, and is calculated with the following formula:

\begin{align*}
\sigma_X=\sqrt{\frac{\sum(x_i-\overline{X})^2}{N}}
\end{align*}

where $x_i$ is the weekly returns value, $\overline{X}$ is the expected returns value, and ${N}$ is the number of values in the dataset. This is an important metric to determine since the standard deviation of portfolio returns is correlated to price fluctuations in the data. Since we want to maximize risk, we want to choose stocks with greater standard deviation values. By understanding how the returns of different stocks deviate from their means, investors can make informed decisions about diversifying their portfolio to achieve a desired level of risk. Here, the function get_return_stds calculates the standard deviation of the weekly returns we already calculated.


In [6]:
# Get standard deviations of the percent returns
def get_return_stds():
  return weekly_returns.std().to_frame().T

return_stds = get_return_stds()
display(return_stds)

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
0,0.029738,0.028111,0.032859,0.033837,0.034998,0.039356,0.03877,0.039015,0.04002,0.028776,...,0.04303,0.02112,0.082321,0.024786,0.028147,0.0315,0.029917,0.031844,0.033851,0.052283


Beta is a measure of a stock's volatility in relation to the overall market. A beta of 1 means that the price of the stock moves with the market, while a negative beta indicates that the stock price moves in the opposite direction in relation to the market. If the beta is greater than 1, we know that the stock’s price moves up and down more significantly in response to market changes. If the beta is less than 1, we know that the stock price moves less extremely in response to the market. Here, the function get_betas determines the beta values for each stock. We then will use these values in our trading strategy.

In [7]:
# Get beta values of the stocks
def get_betas():
  market_ticker = yf.Ticker(MARKET_INDEX)
  market_history = market_ticker.history(start=START_DATE, end=END_DATE)

  # Resample to weekly market data and get percent returns
  market_weekly_returns = market_history.Close.resample("W").last().pct_change().dropna()

  # Calculate beta value
  temp = pd.concat([weekly_returns, market_weekly_returns], axis=1)
  beta = temp.cov().div(temp.var()).Close.drop(index="Close").to_frame().T

  return beta

betas = get_betas()
display(betas)

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Close,1.228699,0.25143,0.803153,1.240952,0.750641,1.289406,1.409978,1.275695,1.450656,0.732595,...,1.705502,0.696267,2.406784,0.459876,0.850252,1.179653,0.030758,1.062848,1.265794,1.541963


Our goal is to create a trading algorithm that creates the most optimal risky portfolio that generates the greatest absolute change in value. To accomplish this, we have taken various factors into consideration. First, we want to maximize or minimize beta. The overall market acts as a baseline to all the stock, so we want to consider the stocks that deviate the most from this. We also want to maximize standard deviation to maximize the volatility, hence maximizing the risk. We want to analyze the highest and lowest betas separately from the standard deviation considering that if standard deviations were the only statistic taken into consideration, the stocks may move opposite to each other. This could cause the portfolio to not deviate as much in value compared to if the stocks made similar movements in the same direction.

We chose to assign the weighting of the riskiest stocks by having the top three worth 20% each, fourth riskiest worth 10%, and the remaining six worth 5% each. We chose to only add 10 stocks to the portfolio to have as least diversification as possible. We want to maximize riskiness in the portfolio so the stocks with the highest standard deviation will be weighted higher than stocks with lower deviation.


In [8]:
# Pick stocks that give the highest standard deviation from the highest or lowest beta values
def select_stocks():
  combined_data = pd.concat([betas.T, return_stds.T], axis=1)
  combined_data.columns = ["betas", "return_stds"]

  # Sort data by beta values
  combined_data = combined_data.sort_values(by="betas", ascending=True)

  # Get 10 lowest and 10 highest beta value stocks and sort them by standard deviation
  lowest_betas = combined_data.head(10).sort_values(by="return_stds", ascending=False)
  highest_betas = combined_data.tail(10).sort_values(by="return_stds", ascending=False)

  # Get total standard deviation of 10 lowest and 10 highest beta value stocks
  lowest_betas_std = lowest_betas.reset_index().betas.mul(STOCKS_WEIGHTING).sum()
  highest_betas_std = highest_betas.reset_index().betas.mul(STOCKS_WEIGHTING).sum()

  # Return list of stocks that yield the highest standard deviation
  return list(highest_betas.index if highest_betas_std > lowest_betas_std else lowest_betas.index)

best_stocks = select_stocks()
print(best_stocks)

['SHOP.TO', 'USB', 'PYPL', 'CAT', 'QCOM', 'BAC', 'AMZN', 'BA', 'AXP', 'BLK']


The following generates the forex rate for USD to CAD for the previous month. Since our cash is in Canadian dollars, we need to convert the shares prices in USD to CAD before adding them to our portfolio later on.

In [9]:
# Get USD to CAD conversion history
usd_to_cad_ticker = yf.Ticker(USD_TO_CAD)
usd_to_cad_history = usd_to_cad_ticker.history()
display(usd_to_cad_history.tail(5))

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-11-21 00:00:00+00:00,1.37241,1.37307,1.36790,1.37241,0,0.0,0.0
2023-11-22 00:00:00+00:00,1.37010,1.37641,1.36904,1.37010,0,0.0,0.0
2023-11-23 00:00:00+00:00,1.36925,1.37100,1.36512,1.36925,0,0.0,0.0
2023-11-24 00:00:00+00:00,1.36950,1.37112,1.35940,1.36950,0,0.0,0.0
2023-11-27 00:00:00+00:00,1.36350,1.36410,1.36210,1.36353,0,0.0,0.0


The function create_portfolio generates the portfolio from the stocks and weightings determined by the algorithm. The overall portfolio value is $\$750,000$ sub the $\$4.95$ transaction fee per ticker. All the shares in USD were converted to Canadian dollars and all transactions were performed in terms of Canadian dollars including the transaction fees. Note the total portfolio value and the total portfolio weighting vary by less than 9 decimals places. This can be attributed to how numbers are stored in Python.

In [10]:
# Create Portfolio_Final
def create_portfolio():
  Portfolio_Final = pd.DataFrame()

  for i, symbol in enumerate(best_stocks):
    weight = STOCKS_WEIGHTING[i]
    value = weight * (STARTING_VALUE - NUM_STOCKS * TRADING_FEE)
    currency = stock_tickers[symbol].fast_info["currency"]
    close_raw = stock_tickers[symbol].history().Close.iat[-1]
    price = close_raw * usd_to_cad_history.Close.iat[-1] if currency == "USD" else close_raw
    shares = value / price
    row = pd.DataFrame([symbol, price, currency, shares, value, weight]).T
    Portfolio_Final = pd.concat([Portfolio_Final, row], axis=0)

  Portfolio_Final.columns = ["Ticker", "Price", "Currency", "Shares", "Value", "Weight"]
  Portfolio_Final.index = [i + 1 for i in range(NUM_STOCKS)]
  return Portfolio_Final

Portfolio_Final = create_portfolio()

# Prove totals add up to the right amounts
print(f"Portfolio Total Value: {Portfolio_Final.Value.sum() + NUM_STOCKS * TRADING_FEE}")
print(f"Portfolio Total Weight: {Portfolio_Final.Weight.sum()}")

display(Portfolio_Final)

Portfolio Total Value: 750000.0000000002
Portfolio Total Weight: 1.0000000000000002


,Ticker,Price,Currency,Shares,Value,Weight
1,SHOP.TO,96.260002,CAD,1558.176778,149990.1,0.2
2,USB,50.723319,USD,2957.024587,149990.1,0.2
3,PYPL,76.030433,USD,1972.763992,149990.1,0.2
4,CAT,337.282791,USD,222.350656,74995.05,0.1
5,QCOM,174.190963,USD,215.266765,37497.525,0.05
6,BAC,40.537747,USD,925.002679,37497.525,0.05
7,AMZN,200.084406,USD,187.408533,37497.525,0.05
8,BA,299.976609,USD,125.001496,37497.525,0.05
9,AXP,224.191607,USD,167.256596,37497.525,0.05
10,BLK,995.513249,USD,37.666525,37497.525,0.05


In [11]:
# Create Stocks_Final and save it as a csv
Stocks_Final = Portfolio_Final[["Ticker", "Shares"]]
Stocks_Final.to_csv(f"Stocks_Group_{TEAM_NUMBER}.csv")

Throughout the assignment, we analyzed historical stock data from the given tickers and ran algorithms to determine the most optimal portfolio for a risky trading strategy. We considered statistics such as standard deviation and beta to support our algorithm. In the end, we generated a portfolio worth \$750,000 with the stocks and weightings determined by our algorithm.  

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Steven Chen, Jolene Huang, Haadi Khan